# Deep Temporal Transformer for Financial Fraud Detection
**Author:** Prasad Kharat  
**GPU Optimized** - Works on any GPU (T4, V100, A100)

This notebook demonstrates the complete fraud detection pipeline.

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install torch torchvision scikit-learn pandas matplotlib seaborn -q
print("✅ Dependencies installed!")

## 2. Upload Your Code

**Option A:** Upload the entire project folder as a zip file, then unzip it.

**Option B:** Clone from GitHub (if you've pushed it):
```python
!git clone https://github.com/yourusername/Deep-Temporal-Transformer.git
%cd Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection
```

**Option C:** Use the file upload below:

In [ ]:
# Option C: Upload zip file
from google.colab import files
import zipfile

print("Upload your project zip file:")
uploaded = files.upload()

# Unzip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✅ Extracted {filename}")

# Navigate to directory
%cd Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection

## 3. Install the Package

In [ ]:
# Install the package in development mode
!pip install -e . -q
print("✅ Package installed!")

## 4. Quick Demo - Basic Usage

In [ ]:
from deep_temporal_transformer import (
    get_default_config, DataProcessor, ModelTrainer,
    set_random_seeds, get_device
)

# Setup
set_random_seeds(42)
config = get_default_config()
device = get_device()

print(f"🚀 Using device: {device}")
print(f"📊 Model config: d_model={config.model.d_model}, layers={config.model.num_layers}")

In [ ]:
# Load and process data (using synthetic data for demo)
processor = DataProcessor(seq_len=8, random_state=42)
X_train, y_train, X_val, y_val, X_test, y_test = processor.process_data()

print(f"✅ Data loaded:")
print(f"  Train: {X_train.shape}")
print(f"  Val: {X_val.shape}")
print(f"  Test: {X_test.shape}")
print(f"  Fraud rate: {y_train.mean():.2%}")

In [ ]:
# Train model (adjust epochs for quick demo)
config.training.epochs = 10  # Reduce for quick demo
config.training.batch_size = 256

trainer = ModelTrainer(config, device)
trainer.setup_model(input_dim=X_train.shape[-1])

print(f"🧠 Model initialized with {sum(p.numel() for p in trainer.model.parameters()):,} parameters")
print("\n🎯 Starting training...")

history = trainer.train(X_train, y_train, X_val, y_val)

In [ ]:
# Evaluate on test set
results = trainer.evaluate_model(X_test, y_test)

print("\n" + "="*60)
print("📊 TEST SET RESULTS")
print("="*60)
print(f"F1 Score:       {results['f1']:.4f}")
print(f"AUC Score:      {results['auc']:.4f}")
print(f"Precision:      {results['precision']:.4f}")
print(f"Recall:         {results['recall']:.4f}")
print(f"Inference Time: {results['avg_inference_time']:.6f}s per transaction")
print("="*60)

## 5. Advanced Usage - GPU-Optimized Model

In [ ]:
from deep_temporal_transformer.models.advanced_transformer import DeepTemporalTransformerAdvanced
from deep_temporal_transformer.training.advanced_training import (
    detect_and_configure_gpu, FocalLossAdvanced
)

# Auto-detect and configure GPU
gpu_config = detect_and_configure_gpu()
device = gpu_config['device']

print("\n🚀 GPU Configuration:")
print(f"  GPU: {gpu_config['gpu_name']}")
print(f"  Memory: {gpu_config['gpu_memory_gb']:.1f} GB")
print(f"  Mixed Precision: {'BF16' if gpu_config['use_bf16'] else ('FP16' if gpu_config['use_fp16'] else 'FP32')}")
print(f"  Batch Size Multiplier: {gpu_config['batch_size_multiplier']:.1f}x")

In [ ]:
# Initialize advanced model with all innovations
model = DeepTemporalTransformerAdvanced(
    input_dim=X_train.shape[-1],
    d_model=256,
    num_heads=8,
    num_layers=6,
    num_experts=8,
    memory_slots=512,
    use_gradient_checkpointing=True  # Enable for memory efficiency
).to(device)

print(f"\n🧠 Advanced Model Initialized")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Features: Sparse Attention + MoE + Temporal Modules + External Memory")

In [ ]:
# Example: Predict with uncertainty estimation
import torch

# Convert test data to tensor
X_test_tensor = torch.FloatTensor(X_test[:100]).to(device)

# Get predictions with uncertainty
mean_probs, uncertainty = model.predict_with_uncertainty(X_test_tensor, n_samples=10)

print("\n🎯 Uncertainty-Aware Predictions (first 5):")
for i in range(5):
    print(f"  Sample {i+1}: Fraud Prob={mean_probs[i]:.4f}, Uncertainty={uncertainty[i]:.4f}")

## 6. Visualizations

In [ ]:
from deep_temporal_transformer.evaluation.explain import ModelExplainer
import matplotlib.pyplot as plt

# Create explainer
explainer = ModelExplainer(trainer.model, device)

# Plot training history
explainer.plot_training_history(history)
plt.show()

# Plot confusion matrix
explainer.plot_confusion_matrix(X_test, y_test)
plt.show()

print("✅ Visualizations complete!")

## 7. Save Model (Optional)

In [ ]:
# Save to Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

# Save model
save_path = '/content/drive/MyDrive/fraud_detection_model.pt'
trainer.save_model(save_path)
print(f"✅ Model saved to: {save_path}")

## 🎉 Complete!

You've successfully:
- ✅ Set up the environment
- ✅ Trained the Deep Temporal Transformer
- ✅ Evaluated performance
- ✅ Explored advanced features
- ✅ Generated visualizations

**Next Steps:**
- Use your own dataset instead of synthetic data
- Experiment with hyperparameters
- Try the advanced model with uncertainty estimation
- Generate thesis plots and results